<a href="https://colab.research.google.com/github/RadhavaramHarika/Harika_INFO5731_Spring2020/blob/master/Harika_INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

In [113]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as panda

def getDataFromUrl(url):
  with urllib.request.urlopen(url) as file:
    web_content = file.read()
    b_soup = BeautifulSoup(web_content)
  return b_soup

def getUserReviewList(urlData):
  review_data = urlData.find_all("div",{"class":"lister-item-content"})
  reviews_list = []
  for each in review_data:
    title = each.find('a').get_text()
    review = each.find(class_ = "content").find(class_ = "text show-more__control").get_text()
    reviews_list.append([title, review])
  return reviews_list

def getHundredUserReviewList(urlData):
  total_reviews = getUserReviewList(urlData)
  count=len(total_reviews)
  base_url = loadMoreReviews(urlData)
  while len(total_reviews)!=100:
    if base_url is not None:
      content = getDataFromUrl(base_url)
      for each in getUserReviewList(content):
        total_reviews.append(each)
    else:
      break
    count = len(total_reviews)
  return total_reviews

def extractFirstReview(reviewList):
  user_review = reviewList[0]
  return user_review

def retriewTitles(reviewList):
  titles = [each.find('a').get_text() for each in reviewList]
  return titles 

def retriewTotalReview(reviewList):
  totalReviews = [each.find(class_ = "content").find(class_ = "text show-more__control").get_text() for each in reviewList]
  return totalReviews 

def loadMoreReviews(url_data):
  moreReviews = url_data.select(".load-more-data") if url_data else None
  if len(moreReviews):
    ajax_url = moreReviews[0]["data-ajaxurl"]
    url = "https://www.imdb.com"+ajax_url+"?ref_=undefined&paginationKey="
    try:    
      key = moreReviews[0]["data-key"]
    except KeyError:
      key = None
  return url+key if key else None

def uploadReviewsToCSV(reviewList):
  csv_dict = {"Review Title":[each[0] for each in reviewList],
              "Total Review":[each[1] for each in reviewList]}
  datafr = panda.DataFrame(csv_dict)
  datafr.to_csv('UserReview.csv', index = False)
  return datafr


url = "https://www.imdb.com/title/tt7286456/reviews?start=0"
web_content = getDataFromUrl(url)

user_reviews = getHundredUserReviewList(web_content)
print(len(user_reviews))

first_review = extractFirstReview(user_reviews)
print(first_review)

uploadReviewsToCSV(user_reviews)

100
[" As a viewer that actually went to TIFF and witnessed this film and didn't want to believe the hype, it is an absolute MASTERPIECE and Phoenix is a certified legend.\n", "I was a person that saw all the hype and claims of masterpiece as overreacting and overblown excitement for another Joker based film. I thought this looked solid at best and even a bit too pretentious in the trailer, but in here to say I was incredibly wrong. This is a massive achievement of cinema that's extremely rare in a day and age of cgi nonsense and reboots. While this is somewhat of a reboot of sorts, the standalone origin tale is impeccable from start to finish and echoes resemblance to the best joker origin comics from the past. Joaquin bleeds, sweats, and cries his every drop into this magnificently dedicated performance. Heath Ledger would be proud. This is undoubtedly the greatest acting performance since Heath's joker. The directing and writing is slickly brilliant and the bleak settings and tones 

,Review Title,Total Review
0,As a viewer that actually went to TIFF and wi...,I was a person that saw all the hype and claim...
1,Outstanding movie with a haunting performance...,"Every once in a while a movie comes, that trul..."
2,Only certain people can relate\n,This is a movie that only those who have felt ...
3,Perfect in every aspect.\n,"Truly a masterpiece, The Best Hollywood film o..."
4,The Hype is real\n,Most of the time movies are anticipated like t...
...,...,...
95,Astonishing Masterpiece\n,What an incredible ride this was. I was almost...
96,Nonsense plot\n,Arthur Fleck lives with his invalid mother. He...
97,I feel like everyone is brain washed\n,I thought this movie was complete and utter ga...
98,I made an account just to rate my disappoint ...,I was expecting a masterpiece and oscar worthy...


In [114]:
import nltk
import csv,os,math,unicodedata,re
from google.colab import drive,files
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import Counter 
from textblob import TextBlob
from nltk.stem import PorterStemmer
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import pandas as panda
from sklearn.feature_extraction.text import TfidfVectorizer
import string,io
from google.colab import drive,files

def remove_punctuations(samp_sent):
  removed_punct = []
  translator = str.maketrans('', '', string.punctuation) 
  for each in samp_sent:
    removed_punct.append(each.translate(translator))
  return removed_punct

def remove_numbers(samp_sent):
  removed_numb = []
  for each in samp_sent:
    removed_numb.append(re.sub(r"\d", "", each))
  return removed_numb

def remove_stop_words(samp_sent):
  stops = stopwords.words('english')
  no_stops = []
  for each in samp_sent:
    words = each.split()
    no_stops.append(' '.join([w for w in words
                              if w not in stops]))
  return no_stops

def lower_sentences(samp_sent):
  lowers = []
  for each in samp_sent:
    lowers.append(each.lower())
  return lowers

def stemming(samp_sent_tokens):
  stem_sent = []
  ps = PorterStemmer()
  for each in samp_sent_tokens:
    words = word_tokenize(each)
    stem_sent.append([ps.stem(w) for w in words])
  return stem_sent

def lemmatized_sent_tokens(samp_sent_tokens):
  lmtzr = WordNetLemmatizer() 
  lem_sent_tokens = []
  for each in samp_sent_tokens:
    lem_sent_tokens.append([lmtzr.lemmatize(w) for w in each])
  return lem_sent_tokens

def uploadPreprocessedDataToCSV(reviews, data):
  no_punct_sent = remove_punctuations(reviews)
  no_stop_sent = remove_stop_words(no_punct_sent)
  no_numb_sent = remove_numbers(no_stop_sent)
  lowered_sent = lower_sentences(no_numb_sent)
  stem_sent_tokens = stemming(lowered_sent)
  lemmatized_tokens = lemmatized_sent_tokens(stem_sent_tokens)
  #[word_tokenize(each) for each in lowered_sent]
  data["Preprocessed Reviews"] = [' '.join(each) for each in lemmatized_tokens]
  data.to_csv('UserReview.csv',index = False)
  
  return data
  
data = panda.read_csv('UserReview.csv')
uploadPreprocessedDataToCSV(data['Total Review'],data)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Review Title,Total Review,Preprocessed Reviews
0,As a viewer that actually went to TIFF and wi...,I was a person that saw all the hype and claim...,i person saw hype claim masterpiec overreact o...
1,Outstanding movie with a haunting performance...,"Every once in a while a movie comes, that trul...",everi movi come truli make impact joaquin perf...
2,Only certain people can relate\n,This is a movie that only those who have felt ...,thi movi felt alon isol truli relat you unders...
3,Perfect in every aspect.\n,"Truly a masterpiece, The Best Hollywood film o...",truli masterpiec the best hollywood film one b...
4,The Hype is real\n,Most of the time movies are anticipated like t...,most time movi anticip like end fall short way...
...,...,...,...
95,Astonishing Masterpiece\n,What an incredible ride this was. I was almost...,what incred ride i almost motionless throughou...
96,Nonsense plot\n,Arthur Fleck lives with his invalid mother. He...,arthur fleck live invalid mother he suffer men...
97,I feel like everyone is brain washed\n,I thought this movie was complete and utter ga...,i thought movi complet utter garbag i dont und...
98,I made an account just to rate my disappoint ...,I was expecting a masterpiece and oscar worthy...,i expect masterpiec oscar worthi film hype i a...


# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [112]:
# Write your code here
from nltk import ngrams,FreqDist,bigrams
import pandas as panda
import string,io,os
from google.colab import drive,files

def calculate_n_grams(data,n):
  n_grams = []
  for each in data['Preprocessed Reviews'][:10]:
    n_grams.append(ngrams(each.split(), n))
  return n_grams

def get_reviewData():
  data = panda.read_csv('UserReview.csv')
  return data

def freq_cal(n_grams):
  freq_ngram = []
  for each in n_grams:
    fdist = nltk.FreqDist(each)
    freq_ngram.append(list(fdist.most_common()))
  return freq_ngram

def freq(n_grams):
  freq_ngram = []
  for each in n_grams:
    fdist = nltk.FreqDist(each)
    freq_ngram.append(fdist)
  return freq_ngram

def count_prob(dataframe):
  data = dataframe['Preprocessed Reviews'][:10]
  uni_grams = [word_tokenize(each) for each in data]
  bi_grams = [list(ngrams(each,2)) for each in uni_grams]
  uni_frq = freq_cal(uni_grams)
  bi_freq = freq_cal(bi_grams)
 
  count_prob_bi = []

  for i in range(len(bi_freq)):
    prob_each = []
    for each in bi_freq[i]:
      c_b = int(each[1])
      w2 = each[0][0]
      for e in uni_frq[i]:
        if e[0]==w2:
          c_w2 = int(e[1])
      prob_each.append(c_b/c_w2)
    count_prob_bi.append(prob_each)
  return count_prob_bi

datafr = get_reviewData()
trigrams = calculate_n_grams(datafr,3)
print("\nQuestion 1: Count all Tri-grams:\n")
print("Tri-gram\t\tCount")
for each in freq_cal(trigrams):
  for gram in each:
    print(str(gram[0])+"\t"+str(gram[1]))

list_prob = count_prob(datafr)
print("\nQuestion 2: Probabilities of all bi-grams:\n")
for each in list_prob:
  print(each)




Question 1: Count all Tri-grams:

Tri-gram		Count
('i', 'person', 'saw')	1
('person', 'saw', 'hype')	1
('saw', 'hype', 'claim')	1
('hype', 'claim', 'masterpiec')	1
('claim', 'masterpiec', 'overreact')	1
('masterpiec', 'overreact', 'overblown')	1
('overreact', 'overblown', 'excit')	1
('overblown', 'excit', 'anoth')	1
('excit', 'anoth', 'joker')	1
('anoth', 'joker', 'base')	1
('joker', 'base', 'film')	1
('base', 'film', 'i')	1
('film', 'i', 'thought')	1
('i', 'thought', 'look')	1
('thought', 'look', 'solid')	1
('look', 'solid', 'best')	1
('solid', 'best', 'even')	1
('best', 'even', 'bit')	1
('even', 'bit', 'pretenti')	1
('bit', 'pretenti', 'trailer')	1
('pretenti', 'trailer', 'say')	1
('trailer', 'say', 'i')	1
('say', 'i', 'incred')	1
('i', 'incred', 'wrong')	1
('incred', 'wrong', 'thi')	1
('wrong', 'thi', 'massiv')	1
('thi', 'massiv', 'achiev')	1
('massiv', 'achiev', 'cinema')	1
('achiev', 'cinema', 'that')	1
('cinema', 'that', 'extrem')	1
('that', 'extrem', 'rare')	1
('extrem', 'rare'

# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [0]:
# Write your code here





# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [0]:
# The GitHub link of your final csv file

# Link: 